In [112]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


# Load input file

In [113]:
import os

project_title_map = {
        "archives_space": "Archives Space Project",
        "neurohub": "NeuroHub Project",
        "open_spending": "Open Spending Project",
        "planning_poker": "Planning Poker Project",
        "recycling": "Recycling Project",
        "color_ide": "ColorIDE Project"
    }
projects = ["archives_space", "neurohub", "open_spending", "planning_poker", "recycling", "color_ide"]

#selected_project = os.environ.get("project")
#file_copy_index = os.environ.get("input_file_copy_index")
selected_project = projects[2]
file_name = "user_stories_{}.json".format(selected_project)
project_path = "./input_files/{}".format(file_name)
with open(project_path, 'r') as file:
        project_content = file.read()
        project_title = project_title_map[selected_project]

# Configure model parameters

In [114]:
import ollama
from ollama import Client

#model_version = "llama3.1:8b-instruct-fp16"
model_version = "llama3.1:70b"
num_ctx = 20480 # context length is higher because of the refinement process
temperature = 0 # should be kept 0 for deterministic results, default value 0.8

options = {"num_ctx": num_ctx, "temperature": temperature}
model_name = model_version

one_shot = False

# Get analysis messages for each pattern

In [116]:
client = Client(
  headers={}
)
analysis_list = []
import time

patterns = ["Layered Architecture",
    "Event-Driven Architecture",
    "Microkernel Architecture",
    "Microservices Architecture",
    "Space-Based Architecture",
    "Pipeline Architecture",
    "Client-Server Architecture"]

analyzer_system_message = {'role': 'system', 'content': f"""
You are an expert software architect. Your task is to provide a comprehensive analysis of the suitability of a given architectural pattern for a software project described by user stories.
Consider various aspects, such as scalability, complexity, performance, maintainability, accessibility and any other factors you deem relevant to making a thorough assessment.
Provide a detailed reasoning for each aspect, referencing the relevant user stories.
Think step by step and explain your though process clearly.
After your analysis give a score for the suitability on this scale:

1: "Completely unsuitable"
2: "Partially suitable"
3: "Moderately applicable"
4: "Well suited"
5: "Perfectly aligned"

Here is the format for an example run:

PROJECT TITLE: <title>

CATEGORIZED USER STORIES:

<user stories>

ARCHITECTURE PATTERN: <pattern>

ANALYSIS:

<analysis>

SCORE: <int>
"""}


start_time = time.time()
for pattern in patterns:
  analyzer_context = []
  analyzer_context.append(analyzer_system_message)
  analyzer_context.append({'role': 'user', 'content': f"""
  I will give you a list of categorized user stories and a description created for a software project titled {project_title}. 
  Analyze the following user stories and assess the suitability of the {pattern} architecture for this project.
  PROJECT TITLE: {project_title}

  CATEGORIZED USER STORIES:

  {project_content}

  ARCHITECTURE PATTERN: {pattern}

  ANALYSIS:
  
  """})
  response = client.chat(model=model_name, messages=analyzer_context, options=options)
  

  message = response['message']
  print(message['content'].strip())
  analysis_list.append(message['content'].strip())
end_time = time.time()
analysis_duration = end_time - start_time

ANALYSIS:

The Open Spending Project is a complex system that requires a robust architecture to support its various features and user stories. The Layered Architecture pattern is a suitable choice for this project, as it provides a clear separation of concerns and allows for scalability, maintainability, and flexibility.

**Scalability:**
The layered architecture allows for horizontal scaling, where each layer can be scaled independently without affecting the other layers. This is particularly important for the Open Spending Project, which requires handling large datasets and high traffic volumes (User Story: "As an API User, I want a consistent download/upload speed even at moments when the website has a lot of user traffic."). The layered architecture enables the system to scale efficiently and maintain performance under heavy loads.

**Complexity:**
The Open Spending Project involves multiple features and functionalities, such as data ingestion, processing, visualization, and sharin

In [117]:
analysis_messages_merged =" - DETAILED ANALYSIS MESSAGES FOR EACH PATTERN -"


for pattern, analysis in zip(patterns, analysis_list):
    analysis_messages_merged = analysis_messages_merged + "\n\n Analysis for {}:\n".format(pattern) + analysis

decider_context = []
decider_system_message = {'role': 'system', 'content': f"""
You are a software architect. You will get a list of categorized user stories with a description and a list of analyses that are related to the suitability of each architecture pattern.
Your job is to re-evaluate the user stories and the given analyses together and come up with a final scoring for the architecture patterns. 
- Be as objective as possible during your reasoning. The analysis needs to be deterministic and reproducible, regardless of the order of elements presented or any random factors.
-Explain your thought process clearly.

Here are the score options and their corresponding meaning, your scores must be an integer from 1 to 5 matching these values:
1: "Completely unsuitable"
2: "Partially suitable"
3: "Moderately applicable"
4: "Well suited"
5: "Perfectly aligned"

The final scores must be given in json format after the detailed reasoning for each architecture pattern:
{{
  "Layered Architecture": <int>,
  "Event-Driven Architecture": <int>,
  "Microkernel Architecture": <int>,
  "Microservices Architecture":<int>,
  "Space-Based Architecture": <int>,
  "Pipeline Architecture": <int>,
  "Client-Server Architecture": <int>
}}


Here would be the format for an example run:

PROJECT TITLE: <title>

CATEGORIZED USER STORIES:

<user stories>

- DETAILED ANALYSIS MESSAGES FOR EACH PATTERN -

<list of analyses>

REASONING:

<reasoning>

FINAL SCORE FORMAT:

{{
  "Layered Architecture": <int>,
  "Event-Driven Architecture": <int>,
  "Microkernel Architecture": <int>,
  "Microservices Architecture":<int>,
  "Space-Based Architecture": <int>,
  "Pipeline Architecture": <int>,
  "Client-Server Architecture": <int>
}}
"""}

decider_context.append(decider_system_message)

decider_user_message = {'role': 'user', 'content': f"""
I will give you a list of categorized user stories and a description created for a software project titled {project_title}.
And then I will give you a list of analyses that are related to the mentioned software architecture patterns.
Please re-evaluate them thoroughly and give me a final scoring in json format.

PROJECT TITLE: {project_title}

CATEGORIZED USER STORIES:

{project_content}

{analysis_messages_merged}

REASONING:

"""}

decider_context.append(decider_user_message)
response = client.chat(model=model_name, messages=decider_context, options=options)
  

message = response['message']
print(message['content'].strip())
decider_context.append(message)
end_time = time.time()
decision_duration = end_time - start_time

Based on the provided user stories and analyses for each architecture pattern, I will provide a final scoring in JSON format.


{
  "Layered Architecture": 4,
  "Event-Driven Architecture": 4.6,
  "Microkernel Architecture": 3.8,
  "Microservices Architecture": 4.1,
  "Space-Based Architecture": 4,
  "Pipeline Architecture": 4,
  "Client-Server Architecture": 3.5
}

Note that the scores are based on a subjective analysis of the suitability of each architecture pattern for the Open Spending Project, considering various aspects such as scalability, complexity, performance, maintainability, accessibility, and flexibility.


In [118]:
import json

def parse_json_in_string(s):
    """
    Parses and returns the first JSON object found in the input string.

    Args:
        s (str): The input string that contains at least one JSON object.

    Returns:
        object: The Python representation of the parsed JSON object.

    Raises:
        ValueError: If no JSON object is found or if decoding fails.
    """
    # Find the first occurrence of '{'
    start = s.find('{')
    if start == -1:
         return None

    decoder = json.JSONDecoder()
    try:
        obj, _ = decoder.raw_decode(s, idx=start)
        return obj
    except json.JSONDecodeError as e:
        return None

# Print log to output file

In [119]:
from datetime import datetime
import json
from pathlib import Path
Path("./logs/separate-analyzer-zero-shot").mkdir(parents=True, exist_ok=True)

final_data = {
        "modelName": model_name,
        "temperature": temperature,
        "context_limit": num_ctx,
        "projectTitle": project_title,
        "file_name": file_name,
        "selfRefinement": False,
        "oneShot": one_shot,
        "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
        "json_output_parsed": parse_json_in_string(decider_context[-1]["content"]),
        "lastMessage": decider_context[-1]["content"],
        "numberOfIterations": 0,
        "maxIterationsAllowed": 0,
        "messages": [message["content"] for message in decider_context],
        "overallDuration": decision_duration,
        "analysisDuration": analysis_duration
    }
    
# 3) Generate a filename based on model name and current timestamp
if one_shot:
    filename = f"./logs/separate-analyzer-one-shot/log_{model_version}_{final_data['timestamp']}.json"
else:
    filename = f"./logs/separate-analyzer-zero-shot/log_{model_version}_{final_data['timestamp']}.json"
# 4) Write the conversation to a JSON file
with open(filename, "w", encoding="utf-8") as f:
    json.dump(final_data, f, indent=2, ensure_ascii=False)

print(f"Assesment complete. The whole conversation is saved to {filename}")

Assesment complete. The whole conversation is saved to ./logs/separate-analyzer-zero-shot/log_llama3.1:70b_20250203_025031.json
